In [ ]:
! pip install firecrawl-py

In [1]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp

In [2]:
app = FirecrawlApp(api_key='xxx')



In [5]:
# Scrape a single URL
url = 'https://www.poppykidsdental.com/post/from-yellow-to-white-expert-tips-for-brightening-your-childs-smile'
scraped_data = app.scrape_url(url)

# # Crawl a website
# crawl_url = 'https://mendable.ai'
# params = {
#     'pageOptions': {
#         'onlyMainContent': True
#     }
# }
# crawl_result = app.crawl_url(crawl_url, params=params)

In [6]:
print(scraped_data)

{'content': '[![The image shows a pair of orange California poppy flowers with blue stems and leaves. Each flower has detailed lines marking the petals and the flower\'s center, suggesting the texture and folds typically seen in poppies. The blue and orange colors provide a complementary contrast, making the image vibrant and visually appealing. The simple and clean illustration style gives the image a cheerful and modern look.](https://assets-global.website-files.com/61d1def902a2a8e195cf22d4/6606321bf53e5029d523339b_icon_poppy_kids.webp)\\\n\\\nPoppy Kids  \\\n\\\nPEDIATRIC DENTISTRY](/)\n\n[ABOUT](/about)\n\n[MEET DR. ANDREA](/about)\n[MEET THE TEAM](/meet-the-team)\n[TOUR THE OFFICE](/tour-the-dentistry)\n\n[SERVICES](/services)\n\n[THE POPPY EXPERIENCE](https://poppykidsdental.com/services#the-poppy-experience)\n[FIRST VISIT](/first-visit)\n[COMPLETE SMILE CARE](http://poppykidsdental.com/services#preventative)\n[PATIENT PORTAL](https://andreaaduna.curveconnex.com/login/?path=%2Fpo

In [ ]:
response = app.scrape_url(
  url='https://www.poppykidsdental.com/post/unraveling-the-importance-of-flossing-for-optimal-oral-health',
  params={
    'extractorOptions': {
      'mode': 'llm-extraction',
      'extractionPrompt': 'Based on the information on the page, extract the information from the schema. ',
      'extractionSchema': {}
    }
  }
)